#### Imports

In [14]:
using Plots
using Random

# Motion Planning Using Convex Optimization

## Model

In [12]:
struct Point
    x::Float64
    y::Float64
end

In [13]:
struct Pose
    position::Point
    orientation::Float64
end

In [50]:
function GenerateRandomConvexPolygon(numberOfVertices::Int64)
    xCoordinateRandomPoints = []
    yCoordinateRandomPoints = []

    for i in 1:numberOfVertices
        append!(xCoordinateRandomPoints, rand())
        append!(yCoordinateRandomPoints, rand())
    end

    sort!(xCoordinateRandomPoints)
    sort!(yCoordinateRandomPoints)

    minimumX = xCoordinateRandomPoints[1]
    maximumX = xCoordinateRandomPoints[numberOfVertices]
    minimumY = xCoordinateRandomPoints[1]
    maximumY = xCoordinateRandomPoints[numberOfVertices]


    xVectors = []
    yVectors = []

    lastTop = minimumX
    lastBot = minimumX;

    for i in 2:numberOfVertices-1
        x = xCoordinateRandomPoints[i];

        if (rand(Bool)) {
            append!(xVectors, x - lastTop)
            lastTop = x;
        } else {
            append!(xVectors, x - lastBot)
            lastBot = x;
        }

    append!(xVectors, maximumX - lastTop)
    append!(xVectors, lastBot - maximumX)

    double lastLeft = minY, lastRight = minY;

    for (int i = 1; i < n - 1; i++) {
        double y = yPool.get(i);

        if (rand(Bool)) {
            yVec.add(y - lastLeft);
            lastLeft = y;
        } else {
            yVec.add(lastRight - y);
            lastRight = y;
        }
    }

    yVec.add(maxY - lastLeft);
    yVec.add(lastRight - maxY);

end


GenerateRandomConvexPolygon (generic function with 1 method)

true

In [51]:
GenerateRandomPoint()

Any[0.2535350471290888, 0.5844415558862293, 0.7620089842815002, 0.7854463770049906, 0.9479028398516919]0.2535350471290888
0.9479028398516919

## Random Map Generation

### Random Convex Polygon Generation

reference: https://cglab.ca/~sander/misc/ConvexGeneration/convex.html

In [7]:
point = Point(1,2)

Point(1.0, 2.0)

In [9]:
point.y

2.0